# FFT accuracy using (py)vkfft
The methodology follows http://www.fftw.org/accuracy/method.html:
* random values are generated with a uniform distribution between -0.5 and 0.5 (for both real and imaginary values)
* the comparison is made with long double precision calculations performed with (py)fftw
* the comparison is made using the norms: $L_n(y) = \left[\Sigma{\left|y\right|^n}\right]^{1/n}$ (n=1,2 or $\infty$)
* the reported average accuracy is $\frac{L_n(fft_{ref} - fft)}{L_n(fft_{ref})}$

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
# pyfftw supports long double accuracy
from pyfftw.interfaces.scipy_fft import fftn as fftwn, ifftn as ifftwn
from scipy.fft import fftn as fftsn, ifftn as ifftsn
from pyvkfft.fft import fftn as vkfftn, ifftn as ivkfftn
from pyvkfft.cuda import VkFFTApp as VkFFTAppcu
from pyvkfft.base import primes
import skcuda.fft as cu_fft

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Init pycuda
cuda_device_name = None
try:
    import pycuda.autoinit
    import pycuda.gpuarray as cua
    has_pycuda = True
    cuda_device_name = pycuda.autoinit.device.name()
    print("Selected CUDA device: ", cuda_device_name)
    
    def fftncu(d):
        dcu = cua.to_gpu(d)
        plan = cu_fft.Plan(d.shape, d.dtype, d.dtype)
        cu_fft.fft(dcu, dcu, plan, scale=False)
        return dcu.get()
    
    def fftnvcu(d):
        dcu = cua.to_gpu(d)
        return vkfftn(dcu).get()

    def fftnvculut(d):
        dcu = cua.to_gpu(d)
        app = VkFFTAppcu(d.shape, d.dtype, useLUT=True)
        return app.fft(dcu).get()

except:
    print("CUDA is not available")
    has_pycuda = False


In [ ]:
# Init pyopencl
cl_device_name = None
try:
    import pyopencl as cl
    import pyopencl.array as cla
    import os
    from pyvkfft.opencl import VkFFTApp as VkFFTAppcl
    
    # Create some context on the first available GPU
    if 'PYOPENCL_CTX' in os.environ:
        ctx = cl.create_some_context()
    else:
        ctx = None
        # Find the first OpenCL GPU available and use it, unless
        for p in cl.get_platforms():
            for d in p.get_devices():
                if d.type & cl.device_type.GPU == 0:
                    continue
                cl_device_name = d.name
                print("Selected OpenCL device: ", d.name)
                ctx = cl.Context(devices=(d,))
                break
            if ctx is not None:
                break
    cq = cl.CommandQueue(ctx)

    def fftnvcl(d):
        dcl = cla.to_device(cq, d)
        return vkfftn(dcl).get()

    def fftnvcllut(d):
        dcl = cla.to_device(cq, d)
        app = VkFFTAppcl(d.shape, d.dtype, queue=cq, useLUT=True)
        return app.fft(dcl).get()

    has_pyopencl = True
except:
    print("OpenCL is not available")
    has_pyopencl = False


In [ ]:
def l1(a,b):
    return abs(a-b).sum() / abs(a).sum()

def l2(a,b):
    return np.sqrt((abs(a-b)**2).sum() / (abs(a)**2).sum())

def li(a,b):
    return abs(a-b).max() / abs(a).max()


## 1D, single precision

In [ ]:
d0 = np.random.uniform(-0.5, 0.5, 2**18) + 1j * np.random.uniform(-0.5, 0.5, 2**18)
d0ld = d0.astype(np.clongdouble)
d0s = d0.astype(np.complex64)

if False:
    # OpenCL
    device_name = "OpenCL: " + cl_device_name
else:
    # CUDA
    device_name = "CUDA: " + cuda_device_name


def accu_1d(n, fft_dic):
    rld = fftwn(d0ld[:n])
    res = {}
    for k,v in fft_dic.items():
        r = v(d0s[:n])
        res[k] = l1(rld, r),l2(rld, r),li(rld, r)
    return res

fft_dic = {"fftw": fftwn}
if has_pycuda:
    fft_dic["vkfft-cuda"] = fftnvcu
    fft_dic["vkfft-cuda-LUT"] = fftnvculut
    fft_dic["cufft"] = fftncu
if has_pyopencl:
    fft_dic["vkfft-opencl"] = fftnvcl
    fft_dic["vkfft-opencl-LUT"] = fftnvcllut


# print(accu_1d(16, fft_dic))

vn, vl1, vl2, vli = [], {}, {}, {}

#print("%7s  %12s  %12s  %12s  %12s"%("N", "vkfft   ", "vkfft-LUT    ", "cufft   ", "fftw   "))
s = "%7s  %14s"%("N", "fftw   ")
r = accu_1d(8, fft_dic)
for k in ["vkfft-cuda", "vkfft-cuda-LUT", "vkfft-opencl", "vkfft-opencl-LUT", "cufft"]:
    if k in r:
        s += "  %14s" % k
print(s)
for n in range(8, len(d0)+1):
    if max(primes(n)) <= 3:
        r = accu_1d(n, fft_dic)
        vn.append(n)
        for k, v in r.items():
            if k not in vl1:
                vl1[k] = []
                vl2[k] = []
                vli[k] = []
            vl1[k].append(v[0])
            vl2[k].append(v[1])
            vli[k].append(v[2])
        s = "%7d  %14e" % (n, vl2["fftw"][-1])
        for k in ["vkfft-cuda", "vkfft-cuda-LUT", "vkfft-opencl", "vkfft-opencl-LUT", "cufft"]:
            if k in vl2:
                s += "  %14e" % vl2[k][-1]
        print(s)


In [ ]:
plt.figure(figsize=(13,1+(len(vl2)+1)*1.5))
vk = vl2.keys()

i=1
for k in vk:
    plt.subplot((len(vl2)+1)//2,2,i)
    plt.semilogx(vn, vl1[k], '-ob', label="L1")
    plt.semilogx(vn, vl2[k], '-ok', label="L2")
    plt.semilogx(vn, vli[k], '-og', label="Linf")
    plt.title(k)
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.xlabel("N", loc='right')
    #plt.ylim(0, 3e-4)
    i+=1


plt.suptitle("1D FFT L2 error (single precision, radix-2,3) - " + device_name)

plt.tight_layout()

plt.figure()
ms = 3

clrs = {"fftw":'-og', "vkfft-cuda": "-^k", "vkfft-cuda-LUT":"-^b", "vkfft-opencl": "-vk", "vkfft-opencl-LUT":"-vb","cufft":"-or"}

for k,v in vl2.items():
    plt.semilogx(vn, v, clrs[k], markersize=ms, label=k)
# plt.semilogx(vn, vl2["vkfft-cuda"], '-ok', markersize=ms, label="vkfft-cuda")
# plt.semilogx(vn, vl2["vkfft-cuda-LUT"], '-ob', markersize=ms, label="vkfft-cuda-LUT")
# plt.semilogx(vn, vl2["fftw"], '-og', markersize=ms, label="fftw")
# plt.semilogx(vn, vl2["cufft"], '-or', markersize=ms, label="cufft")
plt.legend(loc='upper left')
plt.suptitle("1D FFT L2 error (single precision, radix-2,3) - " + device_name)
plt.xlabel("N", loc='right')
plt.grid(True)
plt.tight_layout()

## 2D, single precision

In [ ]:
d0 = np.random.uniform(-0.5, 0.5, (512, 512)) + 1j * np.random.uniform(-0.5, 0.5, (512, 512))
d0ld = d0.astype(np.clongdouble)
d0s = d0.astype(np.complex64)

if False:
    # OpenCL
    device_name = "OpenCL: " + cl_device_name
else:
    # CUDA
    device_name = "CUDA: " + cuda_device_name

def accu_2d(n, fft_dic):
    rld = fftwn(d0ld[:n,:n].copy())
    res = {}
    for k,v in fft_dic.items():
        r = v(d0s[:n,:n].copy())
        res[k] = l1(rld, r),l2(rld, r),li(rld, r)
    return res


vn, vl1, vl2, vli = [], {}, {}, {}

s = "%7s  %14s"%("N", "fftw   ")
r = accu_2d(8, fft_dic)
for k in ["vkfft-cuda", "vkfft-cuda-LUT", "vkfft-opencl", "vkfft-opencl-LUT", "cufft"]:
    if k in r:
        s += "  %14s" % k
print(s)
for n in range(8, len(d0)+1):
    if max(primes(n)) <= 3:
        r = accu_2d(n, fft_dic)
        vn.append(n)
        for k, v in r.items():
            if k not in vl1:
                vl1[k] = []
                vl2[k] = []
                vli[k] = []
            vl1[k].append(v[0])
            vl2[k].append(v[1])
            vli[k].append(v[2])
        
        s = "%7d  %14e" % (n, vl2["fftw"][-1])
        for k in ["vkfft-cuda", "vkfft-cuda-LUT", "vkfft-opencl", "vkfft-opencl-LUT", "cufft"]:
            if k in vl2:
                s += "  %14e" % vl2[k][-1]
        print(s)


In [ ]:
plt.figure(figsize=(13,1+(len(vl2)+1)*1.5))
vk = vl2.keys()

i=1
for k in vk:
    plt.subplot((len(vl2)+1)//2,2,i)
    plt.semilogx(vn, vl1[k], '-ob', label="L1")
    plt.semilogx(vn, vl2[k], '-ok', label="L2")
    plt.semilogx(vn, vli[k], '-og', label="Linf")
    plt.title(k)
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.xlabel("N", loc='right')
    i+=1


plt.suptitle("2D FFT L2 error (single precision, radix-2,3) - " + device_name)

plt.tight_layout()

plt.figure()
ms = 3

clrs = {"fftw":'-og', "vkfft-cuda": "-^k", "vkfft-cuda-LUT":"-^b", "vkfft-opencl": "-vk", "vkfft-opencl-LUT":"-vb","cufft":"-or"}

for k,v in vl2.items():
    plt.semilogx(vn, v, clrs[k], markersize=ms, label=k)
# plt.semilogx(vn, vl2["vkfft-cuda"], '-ok', markersize=ms, label="vkfft-cuda")
# plt.semilogx(vn, vl2["vkfft-cuda-LUT"], '-ob', markersize=ms, label="vkfft-cuda-LUT")
# plt.semilogx(vn, vl2["fftw"], '-og', markersize=ms, label="fftw")
# plt.semilogx(vn, vl2["cufft"], '-or', markersize=ms, label="cufft")
plt.legend(loc='upper left')
plt.suptitle("2D FFT L2 error (single precision, radix-2,3) - " + device_name)
plt.grid(True)
plt.xlabel("N", loc='right')
plt.tight_layout()



# 1D, non-radix (Bluestein) transforms, single precision

In [ ]:
d0 = np.random.uniform(-0.5, 0.5, 512) + 1j * np.random.uniform(-0.5, 0.5, 512)
d0ld = d0.astype(np.clongdouble)
d0s = d0.astype(np.complex64)

if False:
    # OpenCL
    device_name = "OpenCL: " + cl_device_name
else:
    # CUDA
    device_name = "CUDA: " + cuda_device_name


def accu_1d(n, fft_dic):
    rld = fftwn(d0ld[:n])
    res = {}
    for k,v in fft_dic.items():
        r = v(d0s[:n])
        res[k] = l1(rld, r),l2(rld, r),li(rld, r)
    return res

fft_dic = {"fftw": fftwn}
if has_pycuda:
    fft_dic["vkfft-cuda"] = fftnvcu
    fft_dic["vkfft-cuda-LUT"] = fftnvculut
    fft_dic["cufft"] = fftncu
if has_pyopencl:
    fft_dic["vkfft-opencl"] = fftnvcl
    fft_dic["vkfft-opencl-LUT"] = fftnvcllut


# print(accu_1d(16, fft_dic))

vn, vl1, vl2, vli = [], {}, {}, {}

#print("%7s  %12s  %12s  %12s  %12s"%("N", "vkfft   ", "vkfft-LUT    ", "cufft   ", "fftw   "))
s = "%7s  %16s"%("N", "fftw   ")
r = accu_1d(8, fft_dic)
for k in ["vkfft-cuda", "vkfft-cuda-LUT", "vkfft-opencl", "vkfft-opencl-LUT", "cufft"]:
    if k in r:
        s += "  %16s" % k
print(s)
for n in range(8, len(d0)+1):
    if max(primes(n)) >13:  # test only transforms with non-radix dimensions
        r = accu_1d(n, fft_dic)
        vn.append(n)
        for k, v in r.items():
            if k not in vl1:
                vl1[k] = []
                vl2[k] = []
                vli[k] = []
            vl1[k].append(v[0])
            vl2[k].append(v[1])
            vli[k].append(v[2])
        s = "%7d  %16e" % (n, vl2["fftw"][-1])
        for k in ["vkfft-cuda", "vkfft-cuda-LUT", "vkfft-opencl", "vkfft-opencl-LUT", "cufft"]:
            if k in vl2:
                s += "    %14e" % vl2[k][-1]
        print(s)


In [ ]:
plt.figure(figsize=(13,1+(len(vl2)+1)*1.5))
vk = vl2.keys()

i=1
for k in vk:
    plt.subplot((len(vl2)+1)//2,2,i)
    plt.semilogx(vn, vl1[k], '-ob', label="L1")
    plt.semilogx(vn, vl2[k], '-ok', label="L2")
    plt.semilogx(vn, vli[k], '-og', label="Linf")
    plt.title(k)
    plt.grid(True)
    plt.legend(loc='upper left')
    plt.xlabel("N", loc='right')
    i+=1


plt.suptitle("2D FFT L2 error (single precision, Bluestein) - " + device_name)

plt.tight_layout()

plt.figure()
ms = 3

clrs = {"fftw":'-og', "vkfft-cuda": "-^k", "vkfft-cuda-LUT":"-^b", "vkfft-opencl": "-vk", "vkfft-opencl-LUT":"-vb","cufft":"-or"}

for k,v in vl2.items():
    plt.semilogx(vn, v, clrs[k], markersize=ms, label=k)
# plt.semilogx(vn, vl2["vkfft-cuda"], '-ok', markersize=ms, label="vkfft-cuda")
# plt.semilogx(vn, vl2["vkfft-cuda-LUT"], '-ob', markersize=ms, label="vkfft-cuda-LUT")
# plt.semilogx(vn, vl2["fftw"], '-og', markersize=ms, label="fftw")
# plt.semilogx(vn, vl2["cufft"], '-or', markersize=ms, label="cufft")
plt.legend(loc='upper left')
plt.suptitle("2D FFT L2 error (single precision, Bluestein) - " + device_name)
plt.grid(True)
plt.xlabel("N", loc='right')
plt.tight_layout()



In [ ]:
[(n, primes(n)) for n in [491, 492, 493, 494, 496, 497, 498, 499, 501, 502, 503]]